
# Assignment 2 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city. The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals. The locations are divided into three regions: downtown, the suburbs, and the countryside. The following table shows the probabilities of a truck transitioning between these regions at each time step:

| Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside |
|----------------|--------------------------------------------|-----------------------------------------------|------------------------------------------------|
| Downtown       | 0.3                                      | 0.4                                           | 0.3                                            |
| Suburbs        | 0.2                                      | 0.5                                           | 0.3                                            |
| Countryside    | 0.4                                      | 0.3                                           | 0.3                                            |

1. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region after two time steps? [1.5p]
2. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region **the first time** after two time steps? [1.5p]
3. Is this Markov chain irreducible? [1.5p]
4. What is the stationary distribution? [1.5p]
5. Advanced question: What is the expected number of steps until the first time one enters the downtown region having started in the suburbs region. Hint: to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 30. [2p]



In [2]:
# Part 1: Probability of being in downtown after 2 time steps starting from suburbs

import numpy as np

# Transition matrix P (rows: current state, columns: next state)
# States: [downtown, suburbs, countryside]
P = np.array([[0.3, 0.4, 0.3],
              [0.2, 0.5, 0.3],
              [0.4, 0.3, 0.3]])

# Start in suburbs (index 1)
initial_state = np.array([0, 1, 0])

# After 2 steps
P2 = P @ P
# P_large = np.linalg.matrix_power(P, 100)
print(P2)
state_after_2 = initial_state @ P2

problem1_p1 = state_after_2[0]  # Probability of being in downtown
print(f"Part 1: P(downtown after 2 steps | start in suburbs) = {problem1_p1:.4f}")

[[0.29 0.41 0.3 ]
 [0.28 0.42 0.3 ]
 [0.3  0.4  0.3 ]]
Part 1: P(downtown after 2 steps | start in suburbs) = 0.2800


In [3]:
# Part 2: Probability of entering downtown for the FIRST TIME after 2 steps
# This means: step 1 must NOT go to downtown, step 2 must go to downtown

# From suburbs: 
# Step 1: Go to suburbs (0.5) or countryside (0.3) - NOT downtown
# Step 2: From suburbs go to downtown (0.2) OR from countryside go to downtown (0.4)

# Path 1: suburbs -> suburbs -> downtown = 0.5 * 0.2 = 0.1
# Path 2: suburbs -> countryside -> downtown = 0.3 * 0.4 = 0.12

problem1_p2 = 0.5 * 0.2 + 0.3 * 0.4
print(f"Part 2: P(first entry to downtown at step 2 | start in suburbs) = {problem1_p2:.4f}")

Part 2: P(first entry to downtown at step 2 | start in suburbs) = 0.2200


In [ ]:
# Part 3: Is the Markov chain irreducible?
# A chain is irreducible if every state can be reached from every other state

# Transition matrix
P = np.array([[0.3, 0.4, 0.3],
              [0.2, 0.5, 0.3],
              [0.4, 0.3, 0.3]])

# Check if all entries are positive (strongly connected)
# From downtown (row 0): can go to all 3 states
# From suburbs (row 1): can go to all 3 states  
# From countryside (row 2): can go to all 3 states
# So it's irreducible in one step!

problem1_irreducible = True
print(f"Part 3: Is irreducible? {problem1_irreducible}")

In [10]:
# Part 4: Stationary distribution (solve π = π P)

import numpy as np

P = np.array([[0.3, 0.4, 0.3],
              [0.2, 0.5, 0.3],
              [0.4, 0.3, 0.3]])

# Solve π = π P, which is equivalent to π (P - I) = 0
# Or (P^T - I) π^T = 0
# We need to find eigenvector with eigenvalue 1

# eigenvalues, eigenvectors = np.linalg.eig(P.T)

# # Find the eigenvector corresponding to eigenvalue 1
# idx = np.argmax(np.abs(eigenvalues - 1) < 1e-10)
# stationary = np.real(eigenvectors[:, idx])
# stationary = stationary / np.sum(stationary)  # Normalize to sum to 1

# problem1_stationary = stationary
# print(f"Part 4: Stationary distribution = {problem1_stationary}")
# print(f"Sum = {np.sum(problem1_stationary)}")

P_large = np.linalg.matrix_power(P, 100)
stationary = P_large[0, :]  # Any row converges to π
print(stationary)

[0.28888889 0.41111111 0.3       ]


In [13]:
import numpy as np

# 3.8

# States: 0 = Downtown, 1 = Suburbs, 2 = Countryside
P = np.array([
    [0.3, 0.4, 0.3],
    [0.2, 0.5, 0.3],
    [0.4, 0.3, 0.3]
])

def hitting_time(max_steps=30):
    state = 1  # start in Suburbs
    for t in range(1, max_steps + 1):
        state = np.random.choice([0, 1, 2], p=P[state])
        if state == 0:  # Downtown
            return t
    return max_steps  # truncate if not hit

# Run many simulations
N = 100000
times = [hitting_time() for _ in range(N)]

print("Estimated expected steps:", np.mean(times))

Estimated expected steps: 3.856


---
## Assignment 2, PROBLEM 2
Maximum Points = 4


Use the **Multi-dimensional Constrained Optimisation** example (in `07-Optimization.ipynb`) to numerically find the MLe for the mean and variance parameter based on `normallySimulatedDataSamples`, an array obtained by a specific simulation of $30$ IID samples from the $Normal(10,2)$ random variable.

Recall that $Normal(\mu, \sigma^2)$ RV has the probability density function given by:

$$
f(x ;\mu, \sigma) = \displaystyle\frac{1}{\sigma\sqrt{2\pi}}\exp\left(\frac{-1}{2\sigma^2}(x-\mu)^2\right)
$$

The two parameters, $\mu \in \mathbb{R} := (-\infty,\infty)$ and $\sigma \in (0,\infty)$, are sometimes referred to as the location and scale parameters.

You know that the log likelihood function for $n$ IID samples from a Normal RV with parameters $\mu$ and $\sigma$ simply follows from $\sum_{i=1}^n \log(f(x_i; \mu,\sigma))$, based on the IID assumption. 

NOTE: When setting bounding boxes for $\mu$ and $\sigma$ try to start with some guesses like $[-20,20]$ and $[0.1,5.0]$ and make it larger if the solution is at the boundary. Making the left bounding-point for $\sigma$ too close to $0.0$ will cause division by zero Warnings. Other numerical instabilities can happen in such iterative numerical solutions to the MLe. You need to be patient and learn by trial-and-error. You will see the mathematical theory in more details in a future course in scientific computing/optimisation. So don't worry too much now except learning to use it for our problems.  

In [15]:

import numpy as np
from scipy import optimize
# do NOT change the next three lines
np.random.seed(123456) # set seed
# simulate 30 IID samples drawn from Normal(10,2)RV
normallySimulatedDataSamples = np.random.normal(10,2,30)

# define the negative log likelihood function you want to minimize by editing XXX
def negLogLklOfIIDNormalSamples(parameters):
    '''return the -log(likelihood) of normallySimulatedDataSamples with mean and var parameters'''
    mu_param=parameters[0]
    sigma_param=parameters[1]
    
    n = len(normallySimulatedDataSamples)
    sum_sq = np.sum((normallySimulatedDataSamples - mu_param) ** 2)
    
    # Full negative log-likelihood formula with mean and var
    neg_log_lkl = (n / 2) * np.log(2 * np.pi) + n * np.log(sigma_param) + (sum_sq / (2 * sigma_param ** 2))
    return neg_log_lkl 

# you should only change XXX below and not anything else
parameter_bounding_box = ((-20, 20), (0.1, 5.0)) # specify the constraints for each parameter - some guess work...
initial_arguments = np.array([5.0, 1.0]) # point in 2D to initialise the minimize algorithm
result_problem2_opt = optimize.minimize(negLogLklOfIIDNormalSamples, initial_arguments, bounds=parameter_bounding_box, method='L-BFGS-B') 
# call the minimize method above finally! you need to play a bit to get initial conditions and bounding box ok
result_problem2_opt


  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: 58.6313872823858
        x: [ 9.269e+00  1.708e+00]
      nit: 14
      jac: [ 1.208e-05 -2.132e-05]
     nfev: 63
     njev: 21
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

---
## Assignment 2, PROBLEM 3
Maximum Points = 4



Derive the maximum likelihood estimate for $n$ IID samples from a random variable with the following probability density function:
$$
f(x; \lambda) = \frac{1}{24} \lambda^5 x^4 \exp(-\lambda x), \qquad \text{ where, } \lambda>0, x > 0
$$

You can solve the MLe by hand (using pencil paper or using key-strokes). Present your solution as the return value of a function called `def MLeForAssignment2Problem3(x)`, where `x` is a list of $n$ input data points.

In [ ]:

# do not change the name of the function, just replace XXX with the appropriate expressions for the MLe
def MLeForAssignment2Problem3(x):
    '''write comment of what this function does'''
    
    '''Compute the maximum likelihood estimate (MLE) of lambda
       for n IID samples from f(x; lambda) = (1/24) * lambda^5 * x^4 * exp(-lambda*x)
    '''
    
    n = len(x)                          # number of samples
    sum_x = sum(x)                      # sum of all observed x_i
    lambda_hat = 5 * n / sum_x          # MLE formula derived above
    return lambda_hat

---
## Assignment 2, PROBLEM 4
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in ITDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [ ]:

def problem4_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    
    # Parameters: Park–Miller LCG (minimal standard)
    a = 16807
    c = 0
    m = 2**31 - 1  # large prime modulus
    
    # Initialize
    x = seed if seed > 0 else 1  # avoid 0 seed when c = 0
    out = []
    
    # Xn+1​=(aXn​+c) mod m
    
    for _ in range(size):
        x = (a * x + c) % m
        out.append(x / m)  # scale to [0,1)
    
    return out

In [15]:

def problem4_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem1_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    # Generate pseudo-random integers using the provided generator
    raw_values = generator(size=size, seed=seed)
    
    # Scale to [0,1) by dividing by the period
    out = [x / period for x in raw_values]
    
    return out

In [16]:
import numpy as np

def problem4_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution.
    Runs n_iterations
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    n_iterations : an integer denoting how many attempts should be made in the accept-reject sampler
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    
    np.random.seed(seed)
    out = []
    # c = np.pi / 2  # maximum of p0(x) since |sin(2πx)| ≤ 1
    
    # Loop over n_iterations
    for _ in range(n_iterations):
        # draw from proposal q(x) = Uniform(0,1)
        x = uniformGenerator(size=1, seed=np.random.randint(1e9))[0]
        
        # draw acceptance uniform variable
        u = uniformGenerator(size=1, seed=np.random.randint(1e9))[0]
        
        # Acceptance condition: u <= |sin(2πx)|
        if u <= abs(np.sin(2 * np.pi * x)):
            out.append(x)
    
    return out

---
#### Local Test for Assignment 2, PROBLEM 4
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [20]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem4_LCG(size=10, seed = 1))

period = 1

print("Uniform sampler %s" % problem4_uniform(generator=problem4_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem4_uniform(generator=problem4_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator = uniform_sampler,n_iterations=20,seed=1))

LCG output: [7.826369259425611e-06, 0.13153778814316625, 0.7556053221950332, 0.4586501319234493, 0.5327672374121692, 0.21895918632809036, 0.04704461621448613, 0.678864716868319, 0.6792964058366122, 0.9346928959408276]
Uniform sampler [7.826369259425611e-06, 0.13153778814316625, 0.7556053221950332, 0.4586501319234493, 0.5327672374121692, 0.21895918632809036, 0.04704461621448613, 0.678864716868319, 0.6792964058366122, 0.9346928959408276]
Accept-Reject sampler [0.775189422897617, 0.7316238194385655, 0.843403539547419, 0.7103896940641057, 0.3952065340220959, 0.16194159032867364, 0.7255737533446279, 0.7890775384330552, 0.27576229920413453, 0.6327550833266019]


In [21]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [0.8812108472667263, 0.23150374479319968, 0.6913413497806761, 0.2880535475756033, 0.39394369783511585, 0.24610894640740566, 0.09732739592674322, 0.17244817858783146, 0.7468802402469242, 0.21982146498148358, 0.2900291500015505, 0.3398939509411779, 0.26172552288324824]
